In [86]:
import json

# create a new scene graph
def new_scene(name):
    # create empty neutrino data
    data = {
        "meta": {
            "name": ("name", name),
            "scale": ("float", 1.0),
            "asset_path": ("path", "./"),
        },
        "graph": {
            "scene": {},
            "assets": {}
        },
        "internal": {
            "max_key": 0
        }
    }

    # return that empty data
    return data

# write the data to a JSON file
def save_scene(data):
    # create working copy of the scene data
    clean_data = data.copy()

    # get rid of internal data (not to be exported)
    del clean_data["internal"]
    
    filename = data["meta"]["name"][1].replace(" ", "") + ".json"
    with open(filename, "w") as outfile:
        json.dump(clean_data, outfile, indent = 4)

# get a new indexed object key and increment the scene's max key
def add_key(data):
    # get the indexed key
    key = hex(data["internal"]["max_key"] + 1)

    # index the max key
    data["internal"]["max_key"] += 1

    return key

# add an asset to the graph
def add_asset(data, name):
    asset_data = {
        "name": ("name", name)
    }
    
    # add the asset to the graph
    data["graph"]["assets"][add_key(data)] = ("asset", asset_data)

# add an object to the scene
def spawn_object(data, name, asset):
    object_data = {
        "name": ("name", name),
        "asset": ("asset", asset),
        "trans": ("trans", [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [1.0, 1.0, 1.0]])
    }

    # get an asset key by the provided name
    for key, value in data["graph"]["assets"].items():
        if value[1]["name"] == asset:
            object_data["asset"] = ("asset", f"*{key}")

    # add the object to the scene
    data["graph"]["scene"][add_key(data)] = ("object", object_data)

In [87]:
# cache the scene
def cache_scene(data):
    # add the cache object to the scene data
    data["cache"] = {}

    containers = [
        data["graph"]["scene"],
        data["graph"]["assets"]
    ]

    for objects in containers:
        # temp cache
        hash_cache = {}

        # hash all values
        for key, value in objects.items():
            for key, value in value[1].items():
                # ignore pointers
                if type(value) == str:
                    is_pointer = value[0] == "*"
                else:
                    is_pointer = False
                if not is_pointer:
                    # convert into string and hash that
                    value_hash = hash(str(value))

                    # track in temp cache
                    if value_hash not in hash_cache:
                        hash_cache[value_hash] = {"value": value, "count": 1}
                    else:
                        hash_cache[value_hash]["count"] += 1

        # throw out all non-repeated values
        bad_keys = []
        for key, value in hash_cache.items():
            if value["count"] < 2:
                bad_keys.append(key)
        for key in bad_keys:
            del hash_cache[key]

        # create hash objects for each repeated value
        for key, value in hash_cache.items():
            cache_pointer = f"#{add_key(data)}"
            data["cache"][cache_pointer] = value["value"]
            hash_cache[key]["pointer"] = cache_pointer

        # replace all instances of cached values in the graph with corresponding cache pointers
        for object_key, object_value in objects.items():
            for value_key, value_value in object_value[1].items():
                # ignore pointers
                if type(value_value) == str:
                    is_pointer = value_value[0] == "*"
                else:
                    is_pointer = False
                if not is_pointer:
                    # convert into string and hash that
                    value_hash = hash(str(value_value))

                    # if this value is cached, replace it with its cache pointer
                    if value_hash in hash_cache:
                        objects[object_key][1][value_key] = hash_cache[value_hash]["pointer"]

In [88]:
# just returns a random string
import random
import string
def random_string(length):
    return ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(length))

# create test scene
test_scene = new_scene("Neutrino Test Scene")

# populate assets
asset_names = []
for i in range(3):
    name = random_string(8)
    add_asset(test_scene, name)
    asset_names.append(name)

# populate objects in scene
for i in range(5):
    spawn_object(test_scene, random_string(8), random.choice(asset_names))

# cache the scene
cache_scene(test_scene)

save_scene(test_scene)